## 심리상담 챗봇 데이터셋 만들기
- Sentence-Transformer 모델을 사용하여 질문을 변환하여 데이터 셋에 저장
- 프로그램 설치
    - conda activate base; pip install sentence-transfomers
    - 코사인 유사도를 이용하여 입력한 질문과 유사한 질문을 데이터셋에서 찾아서 결과를 내보내줌

In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=65dfc7c0116597ccb8d1b1416dc1b28409583ebdc694c7d5d536f6822dba408b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
from google.colab import files
up = files.upload()

Saving wellness_dataset_original.csv to wellness_dataset_original.csv


In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

#### 1. Sentens BERT 모델 로드

In [9]:
# 한글 문장 임베딩에 활용하기 위해 만들어진 한글 사전학습 모델
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [10]:
# 모델 테스트
sentences = ['안녕하세요?', '한글 문장 임베딩을 위한 BERT 모델입니다.']
embeddings = model.encode(sentences)
print(embeddings)

[[-0.37510446 -0.7733841   0.5927711  ...  0.5792351   0.32683456
  -0.6508966 ]
 [-0.03811122  0.209551    0.02041072 ...  0.25595424  0.27676162
   0.06661456]]


In [11]:
# 한글 문장을 768개의 실수값으로 변환해줌
embeddings.shape

(2, 768)

#### 데이터셋 로드

In [49]:
# 웰니스 데이터셋
df = pd.read_csv('wellness_dataset.csv')
df.head()

,구분,유저,챗봇,embdding,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,[],[]
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,[],[]
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",[],[]
3,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,[],[]
4,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,[],[]


In [50]:
df.tail()

,구분,유저,챗봇,embdding,embedding
1029,치료이력/응급실,"그 사람이 응급실 의사한테 뭐라고 속닥거리니까, 저보고 갑자기 응급처치 끝났다고, ...",응급실이 있어서 다행이네요. 큰 문제는 없으신 거죠?,NaN,NaN
1030,치료이력/응급실,파편이 튀어서 그 때 저도 응급실 가서 치료 받기도 했고…,응급실에 가셨다니 정말 놀랐어요. 아무 문제 없으신가요? 걱정 되네요.,NaN,NaN
1031,현재상태/증상악화,지금 상태가 너무 안 좋아서 학교 안 나가고 있어요.,상태가 더 안 좋아지셨군요. 걱정이 되네요.,NaN,NaN
1032,현재상태/증상악화,진짜 심해진 거 같긴 해요.,정말 힘드시겠어요. 지금도 증상이 심하신가요?,NaN,NaN
1033,현재상태/증상악화,그런데 증상이 나빠진 거 같아.,너무 심하시면 병원을 다시 가보는 건 어떨까요?,NaN,NaN


In [17]:
df.drop(columns=['Unnamed: 3'], inplace=True)
df.dropna(inplace=True)
df.tail()

,구분,유저,챗봇
5196,치료이력/응급실,"그 사람이 응급실 의사한테 뭐라고 속닥거리니까, 저보고 갑자기 응급처치 끝났다고, ...",응급실이 있어서 다행이네요. 큰 문제는 없으신 거죠?
5197,치료이력/응급실,파편이 튀어서 그 때 저도 응급실 가서 치료 받기도 했고…,응급실에 가셨다니 정말 놀랐어요. 아무 문제 없으신가요? 걱정 되네요.
5213,현재상태/증상악화,지금 상태가 너무 안 좋아서 학교 안 나가고 있어요.,상태가 더 안 좋아지셨군요. 걱정이 되네요.
5214,현재상태/증상악화,진짜 심해진 거 같긴 해요.,정말 힘드시겠어요. 지금도 증상이 심하신가요?
5215,현재상태/증상악화,그런데 증상이 나빠진 거 같아.,너무 심하시면 병원을 다시 가보는 건 어떨까요?


In [18]:
df.shape

(1034, 3)

#### 3. 인코딩 후 데이터셋에 저장

In [19]:
# 고정 768개
model.encode(df.유저[0])

array([-4.80606765e-01, -2.94868946e-01,  4.37900156e-01, -6.40137613e-01,
        3.28670964e-02, -3.42647552e-01, -5.47483377e-02,  1.73057280e-02,
       -4.08220828e-01, -5.06033957e-01, -1.68733463e-01, -3.98677081e-01,
       -1.24776639e-01, -9.71539170e-02, -1.65286735e-01,  5.72613720e-03,
        6.13689423e-02, -1.91312224e-01,  2.53917724e-01, -5.85019410e-01,
       -2.84425795e-01, -2.32034847e-01, -3.27080905e-01,  6.72979578e-02,
       -1.64679113e-05, -4.72336322e-01, -3.60021740e-01,  2.91879892e-01,
       -6.63861334e-01, -3.10575008e-01,  5.79525113e-01, -3.11722755e-01,
        1.47699565e-02, -2.12172627e-01,  2.22057983e-01, -1.73829049e-01,
       -3.78458530e-01, -4.20398384e-01, -2.38218650e-01,  6.38705343e-02,
       -1.15304306e-01, -2.44564191e-01, -5.00228345e-01,  1.68355361e-01,
       -6.58360183e-01, -8.91942143e-01, -6.26957238e-01, -3.21965545e-01,
       -7.05358326e-01,  3.71447295e-01, -5.45803249e-01,  7.76299387e-02,
        1.09864689e-01,  

In [43]:
df['embedding'] = pd.Series([[]] * len(df))  #dummy
df.shape

(1034, 5)

In [51]:
df.embedding = df.유저.map(lambda x:list(model.encode(x)))
df.head(3)

,구분,유저,챗봇,embdding,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,[],"[-0.48060676, -0.29486895, 0.43790016, -0.6401..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,[],"[-1.1561577, -0.14506271, 0.2949035, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",[],"[-0.6652, -0.081268415, 1.0945565, 0.105792135..."


In [57]:
df.drop(columns=['embdding'],inplace=True)

In [58]:
df.to_csv('wellness_dataset.csv', index=False)

#### 4. 저장된 데이터셋을 읽고 간단한 챗봇 만들기

In [60]:
import json
wdf = pd.read_csv('wellness_dataset.csv')
wdf.embedding = wdf.embedding.apply(json.loads)
wdf.head(3)

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060676, -0.29486895, 0.43790016, -0.6401..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561577, -0.14506271, 0.2949035, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652, -0.081268415, 1.0945565, 0.105792135..."


In [71]:
# 사용자가 입력한 문장을 임베딩 백터로 인코딩
text = '요즘 골치가 너무 아프고 힘들어'
embedding = model.encode(text)

In [72]:
# 사용자가 입력한 데이터와 데이터셋에 있는 임베딩의 코사인 유사도 계산
wdf['유사도'] = df.embedding.map(lambda x: cosine_similarity([embedding],[x]).squeeze())
# squeeze() 메소드는 리스트 형태의 값을 스칼라 값으로 변환 해줌
wdf.head(3)

,구분,유저,챗봇,embedding,유사도
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060676, -0.29486895, 0.43790016, -0.6401...",0.475140
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561577, -0.14506271, 0.2949035, -0.673949...",0.463033
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652, -0.081268415, 1.0945565, 0.105792135...",0.374445


In [73]:
# 유사도의 값이 최대일 때의 인덱스
wdf.유사도.idxmax()

268

In [74]:
# 챗봇이 사용자에게 제공하는 답변
answer = wdf.챗봇[wdf.유사도.idxmax()]
answer

'정말 힘들고 스트레스 받으시겠어요. 따뜻한 물에 샤워를 추천해드릴게요.'

In [75]:
# 질문과 답변
text = '시험을 망쳐서 미치겠어요'
embedding = model.encode(text)
wdf['유사도'] = df.embedding.map(lambda x: cosine_similarity([embedding],[x]).squeeze())
answer = wdf.챗봇[wdf.유사도.idxmax()]
answer

'운이 나빴던 거라고 생각해요.'